<a href="https://colab.research.google.com/github/Hareg2024/User-churn/blob/main/waze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to bigquery

In [5]:
#import libraries
from google.cloud import bigquery
from google.colab import auth

#authonticate
auth.authenticate_user()

#Initialize the client from bigquery
PROJECT_id = 'userchurn-431116'
client =bigquery.Client(project= PROJECT_id, location='US')


In [16]:
#get the dataset and table , one table at a time
dataset_ref = client.dataset('wazedata', project='userchurn-431116')
table_ref = dataset_ref.table('rawdata')
table = client.get_table(table_ref)
table.schema

[SchemaField('ID', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('label', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('sessions', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('drives', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_sessions', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('n_days_after_onboarding', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_navigations_fav1', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_navigations_fav2', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('driven_km_drives', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('duration_minutes_drives', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('activity_days', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('driving_days', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('device', 'STRING', 'NULLABLE', None, None, (), None)]

In [12]:
# bring the second table for the test table
table_ref = dataset_ref.table('testdata')
testtable = client.get_table(table_ref)
testtable.schema

[SchemaField('ID', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('label', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('sessions', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('drives', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_sessions', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('n_days_after_onboarding', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_navigations_fav1', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('total_navigations_fav2', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('driven_km_drives', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('duration_minutes_drives', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('activity_days', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('driving_days', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('device', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('typ

In [51]:
# convert the 2 tables into dataframe
df = client.list_rows(table=table).to_dataframe()
df.head()


,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device
0,1806,retained,0,0,37.168690,3098,236,0,5570.745476,943.950161,6,0,Android
1,3451,retained,0,0,210.090088,2873,210,35,12012.320180,4804.667454,1,0,Android
2,6375,retained,0,0,99.767601,1386,21,0,4169.721686,3154.856024,2,0,Android
3,8122,retained,0,0,132.322350,89,174,104,5044.893956,2205.094060,4,0,iPhone
4,10586,churned,0,0,222.107433,980,0,6,4932.029466,2656.000047,3,0,Android


In [33]:
# the second table is a view table and cannot be directly converted to dataframe , we are gonna query the view table
sql = """
SELECT *
FROM `userchurn-431116.wazedata.testdata`
"""

query_job = client.query(sql)
results = query_job.result()  # Waits for the query to finish
df_test = results.to_dataframe()
df_test.head()

,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device,type
0,1806,retained,0,0,37.168690,3098,236,0,5570.745476,943.950161,6,0,Android,test
1,3451,retained,0,0,210.090088,2873,210,35,12012.320180,4804.667454,1,0,Android,test
2,6375,retained,0,0,99.767601,1386,21,0,4169.721686,3154.856024,2,0,Android,test
3,8122,retained,0,0,132.322350,89,174,104,5044.893956,2205.094060,4,0,iPhone,test
4,10586,churned,0,0,222.107433,980,0,6,4932.029466,2656.000047,3,0,Android,test


# Build Model

## Install pycaret to access auto ml

In [23]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 94.0 MB/s eta 0:00:00
   ━━

# Code and Train

In [27]:
#get the classification model: user chaurn is catagorical y
from pycaret.classification  import *

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       14999 non-null  Int64  
 1   label                    14299 non-null  object 
 2   sessions                 14999 non-null  Int64  
 3   drives                   14999 non-null  Int64  
 4   total_sessions           14999 non-null  float64
 5   n_days_after_onboarding  14999 non-null  Int64  
 6   total_navigations_fav1   14999 non-null  Int64  
 7   total_navigations_fav2   14999 non-null  Int64  
 8   driven_km_drives         14999 non-null  float64
 9   duration_minutes_drives  14999 non-null  float64
 10  activity_days            14999 non-null  Int64  
 11  driving_days             14999 non-null  Int64  
 12  device                   14999 non-null  object 
dtypes: Int64(8), float64(3), object(2)
memory usage: 1.6+ MB


In [52]:
#handle missing values and duplicates
df=df.dropna(axis=0)

#df=-df.drop_duplicates()

df.duplicated().sum()

0

In [53]:
# setup model
setup(data=df,
      target='label',
      session_id=123,
      ignore_features=['ID','type'],
      categorical_features=['device'])

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Target mapping,"churned: 0, retained: 1"
4,Original data shape,"(14299, 13)"
5,Transformed data shape,"(14299, 12)"
6,Transformed train set shape,"(10009, 12)"
7,Transformed test set shape,"(4290, 12)"
8,Ignore features,2
9,Numeric features,10


In [54]:
#compare all classification model with sklearn models based on evaluation metrics
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8250,0.7487,0.8250,0.7825,0.7723,0.1204,0.1747,2.6180
lr,Logistic Regression,0.8241,0.7573,0.8241,0.7773,0.7676,0.1005,0.1540,0.7690
ada,Ada Boost Classifier,0.8237,0.7382,0.8237,0.7804,0.7768,0.1415,0.1868,0.6100
lda,Linear Discriminant Analysis,0.8237,0.7572,0.8237,0.7759,0.7663,0.0955,0.1486,0.1210
ridge,Ridge Classifier,0.8231,0.7572,0.8231,0.7658,0.7445,0.0073,0.0385,0.1200
dummy,Dummy Classifier,0.8227,0.5000,0.8227,0.6768,0.7426,0.0000,0.0000,0.1990
rf,Random Forest Classifier,0.8201,0.7250,0.8201,0.7747,0.7755,0.1390,0.1777,2.5780
lightgbm,Light Gradient Boosting Machine,0.8200,0.7303,0.8200,0.7733,0.7735,0.1298,0.1697,1.5730
et,Extra Trees Classifier,0.8172,0.7265,0.8172,0.7654,0.7690,0.1117,0.1478,1.2690
xgboost,Extreme Gradient Boosting,0.8121,0.7030,0.8121,0.7684,0.7775,0.1580,0.1805,0.3420


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

### The GradientBoosting model is selected based on evaluation metrics,
* Accuracy= 0.82,Recall(how many of the actual positives were correctly identified)=0.82  and
* Precission (the accuracy of positive predictions)=0.78

In [55]:
# use the best model
gbst = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8312,0.7585,0.8312,0.7990,0.7819,0.1561,0.2199
1,0.8312,0.7403,0.8312,0.7986,0.7832,0.1615,0.2231
2,0.8292,0.7702,0.8292,0.7961,0.7728,0.1177,0.1877
3,0.8222,0.7349,0.8222,0.7734,0.7710,0.1141,0.1590
4,0.8182,0.7499,0.8182,0.7608,0.7624,0.0837,0.1234
5,0.8172,0.7525,0.8172,0.7553,0.7590,0.0696,0.1066
6,0.8242,0.7520,0.8242,0.7803,0.7715,0.1196,0.1724
7,0.8182,0.7050,0.8182,0.7700,0.7737,0.1342,0.1692
8,0.8332,0.7622,0.8332,0.8112,0.7800,0.1509,0.2312


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [56]:
# use the model to predict on holdout data
gbst_pred = predict_model(gbst)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8240,0.7378,0.8240,0.7786,0.7699,0.1105,0.1633


In [58]:
df_final=gbst_pred.copy()
df_final.head(20)

,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device,label,prediction_label,prediction_score
1487,126,110,164.833176,2486,432,0,2159.167725,501.427673,6,1,iPhone,churned,retained,0.8411
6682,9,9,440.216736,1638,139,32,1877.869507,1016.116577,18,11,iPhone,retained,retained,0.9193
7663,155,127,286.282410,782,721,4,6827.522949,3161.975342,12,12,Android,churned,churned,0.6027
12677,68,55,119.666885,957,34,25,1369.640503,402.879852,27,21,Android,retained,retained,0.9570
3436,34,28,136.838440,841,393,10,3064.986328,2048.386719,9,5,iPhone,retained,retained,0.5673
234,21,17,21.705410,955,237,0,4941.263184,4039.858154,2,0,iPhone,churned,churned,0.5077
13280,131,131,170.967514,3185,191,191,4903.722656,3332.038574,22,22,iPhone,retained,retained,0.9472
5637,30,25,36.760063,3436,157,69,3589.366455,1680.339111,16,9,iPhone,churned,retained,0.9080
7278,15,15,29.705603,85,61,41,629.386047,476.107086,18,12,iPhone,retained,retained,0.8765
12796,129,104,131.545685,1755,119,5,9598.928711,2376.764648,29,21,Android,retained,retained,0.9464


In [63]:
churned= df_final['label']
churned.value_counts()


,count
label,
retained,3529
churned,761


In [64]:
churned= df_final['prediction_label']
churned.value_counts()

,count
prediction_label,
retained,4154
churned,136


* The model predicts only 136 churn while it is actually 761 customer churn
* Applying feature engineering might improve the accuracy
* It is also due to feature imbalace , rescaling the data could help improve the accuracy

In [74]:
# Test the model using test data
df_test=df_test.dropna(axis=0)
test =predict_model(gbst, data=df_test)
test.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.7010,0.6876,0.7010,0.6663,0.6408,0.1326,0.1704


,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device,type,label,prediction_label,prediction_score
0,1806,0,0,37.168690,3098,236,0,5570.745605,943.950134,6,0,Android,test,retained,retained,0.8072
1,3451,0,0,210.090088,2873,210,35,12012.320312,4804.667480,1,0,Android,test,retained,retained,0.7543
2,6375,0,0,99.767601,1386,21,0,4169.721680,3154.855957,2,0,Android,test,retained,retained,0.5627
3,8122,0,0,132.322357,89,174,104,5044.894043,2205.093994,4,0,iPhone,test,retained,retained,0.5533
4,10586,0,0,222.107437,980,0,6,4932.029297,2656.000000,3,0,Android,test,churned,retained,0.5184


In [72]:
#actual
churned= test['label']
churned.value_counts()

,count
label,
retained,67
churned,30


In [75]:
# model output
churned= test['prediction_label']
churned.value_counts()

,count
prediction_label,
retained,88
churned,9
